In [30]:
import networkx as nx
import sqlite3
import pandas as pd

conn = sqlite3.connect('/Users/Admin/Documents/Projects/Kairos/main/data/GTFS.db')

In [31]:
MG = nx.MultiDiGraph() # initialise Graph

In [32]:
## Add Nodes

In [33]:
stops = pd.read_sql_query('SELECT * FROM gtfs_stops', conn)
stops.head(1)

,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,stop_code,stop_street,stop_city,stop_region,stop_postcode,stop_country,location_type,parent_station,stop_timezone
0,18848,Clínicas,None,-23.55419,-46.670723,None,None,None,None,None,None,None,None,None,None,None


In [34]:
for i, row in stops.iterrows():
    MG.add_node(row['stop_id'], location=(row['stop_lon'], row['stop_lat']))

## Add Edges

In [38]:
def unicode_to_seconds(a):
    a = map(int, a.split(':'))
    return a[0] * 3600 + a[1] * 60 + a[2]

In [39]:
stops_time = pd.read_sql_query('SELECT * FROM gtfs_stop_times', conn)
stops_time.head(1)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint,arrival_time_seconds,departure_time_seconds
0,1012-10-0,03:40:00,03:40:00,301708,1,None,None,None,None,None,13200,13200


In [40]:
for i, row in stops_time.iterrows():
    
    try:
        partida, chegada = stops_time.iloc[i], stops_time.iloc[i + 1]
    except:
        break
    if chegada['trip_id'] == partida['trip_id']:
        MG.add_edge(partida['stop_id'], chegada['stop_id'], 
                    attr = {'tempo': chegada['departure_time_seconds']-partida['arrival_time_seconds'],
                      'trip_id': chegada['trip_id']}
        )

In [43]:
MG.edge['6415308']

{u'6415312': {0: {'attr': {'tempo': 87, 'trip_id': u'8199-10-0'}}}}

## Add frequency

In [45]:
frequencies = pd.read_sql_query('SELECT * FROM gtfs_frequencies', conn)

In [46]:
frequencies.head(70)

,trip_id,start_time,end_time,headway_secs,start_time_seconds,end_time_seconds
0,1012-10-0,03:00:00,03:59:00,1800,10800,14340
1,1012-10-0,04:00:00,04:59:00,420,14400,17940
2,1012-10-0,05:00:00,05:59:00,420,18000,21540
3,1012-10-0,06:00:00,06:59:00,480,21600,25140
4,1012-10-0,07:00:00,07:59:00,420,25200,28740
5,1012-10-0,08:00:00,08:59:00,480,28800,32340
6,1012-10-0,09:00:00,09:59:00,420,32400,35940
7,1012-10-0,10:00:00,10:59:00,480,36000,39540
8,1012-10-0,11:00:00,11:59:00,420,39600,43140
9,1012-10-0,12:00:00,12:59:00,480,43200,46740


In [47]:
stops_time.head(50)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint,arrival_time_seconds,departure_time_seconds
0,1012-10-0,03:40:00,03:40:00,301708,1,None,None,None,None,None,13200,13200
1,1012-10-0,03:42:00,03:42:00,30003039,2,None,None,None,None,None,13320,13320
2,1012-10-0,03:44:00,03:44:00,30003021,3,None,None,None,None,None,13440,13440
3,1012-10-0,03:46:00,03:46:00,30003019,4,None,None,None,None,None,13560,13560
4,1012-10-0,03:48:00,03:48:00,30003017,5,None,None,None,None,None,13680,13680
5,1012-10-0,03:50:00,03:50:00,301672,6,None,None,None,None,None,13800,13800
6,1012-10-0,03:52:00,03:52:00,301671,7,None,None,None,None,None,13920,13920
7,1012-10-0,03:54:00,03:54:00,301701,8,None,None,None,None,None,14040,14040
8,1012-10-0,03:56:00,03:56:00,301700,9,None,None,None,None,None,14160,14160
9,1012-10-0,03:58:00,03:58:00,301697,10,None,None,None,None,None,14280,14280


In [16]:
stops_time_first = stops_time[stops_time['stop_sequence'] == 1].reset_index(drop=True)
stops_time_first.head(2)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint,arrival_time_seconds,departure_time_seconds
0,1012-10-0,03:40:00,03:40:00,301708,1,None,None,None,None,None,13200,13200
1,1012-10-1,00:50:00,00:50:00,301729,1,None,None,None,None,None,3000,3000


In [48]:
for i, row in stops_time_first.iloc[:5].iterrows():
    
    print 'id', i

    lista = []
    lista.append(row['arrival_time_seconds'])
    trip = frequencies[frequencies['trip_id'] == row['trip_id']]
    
    rodou = 0
    fullday = 0
    
    if len(trip[(trip["start_time_seconds"] <= lista[0]) &
                             (trip["end_time_seconds"] + 60 >= lista[0])]) > 0:
    
        # Para baixo
        while 1:

            ultimo = lista[-1] 
            timetable = trip[(trip["start_time_seconds"] <= ultimo) &
                             (trip["end_time_seconds"] + 60 >= ultimo)]

            if len(timetable) > 0:
                
                soma = timetable['headway_secs'].values[0] + ultimo
                
                if rodou == 1 and soma > lista[0]:
                    fullday = 1
                    print 'fullday'
                    break
                
                else:
                    if soma <= 86340:
                        lista.append(soma)
                    else: 
                        rodou = 1
                        lista.append(soma - 86340)
            
            else:
                break


        # Para cima  
        while fullday==0:

            primeiro = lista[0]
            timetable_atual = trip[(trip["start_time_seconds"] <= primeiro) 
                                   & (trip["end_time_seconds"] + 60 >= primeiro)]

            if timetable_atual['start_time_seconds'].values[0] == 0:
                timetable_anterior = trip[trip['end_time_seconds'] == 82740]
            else:
                timetable_anterior = trip[(trip['end_time_seconds']) == 
                                          timetable_atual['start_time_seconds'].values[0] - 60]

            # com timetable anterior
            if len(timetable_anterior) > 0:

                subt = primeiro - timetable_anterior['headway_secs'].values[0]
                if subt < 0:
                    subt = subt + 82740

                # se dentro da timetable anterior
                if subt <= timetable_anterior['end_time_seconds'].values[0]:
                    lista.insert(0, subt) 
                else:
                    lista.insert(0, primeiro - timetable_atual['headway_secs'].values[0])

            # se dentro da timetable atual mas timetable anterior inexistente 

            elif ((primeiro - timetable_atual['headway_secs'].values[0]) >=
                    timetable_atual['start_time_seconds'].values[0]):

                lista.insert(0, primeiro - timetable_atual['headway_secs'].values[0])

            else:
                break


        if 'viagens' in MG.node[row['trip_id']]:
            MG.node['301729']['viagens'][row['trip_id']] = lista
        else:
            MG.node['301729']['viagens'] = {row['trip_id']: lista}

    else:
        print 'pulou'

id 0


KeyError: u'1012-10-0'

In [29]:
MG.node['6415308']

{'location': {'lat': -23.496564, 'long': -46.711499}}

In [21]:
frequencies[frequencies['trip_id'] == '9005-42-1']

,trip_id,start_time,end_time,headway_secs,start_time_seconds,end_time_seconds
36203,9005-42-1,17:00:00,17:59:00,3600,61200,64740
36204,9005-42-1,18:00:00,18:59:00,1800,64800,68340
